# Tensorboard

- 指定面板图上的变量
    - 是summary
    - 通常merge_all起来 也可以[,]单独指定(e.g.测试时与训练时统计的量不同)
- 计算变量并且保存到文件中
    - 指定到文件夹 会自动创建
    - 需要创建文件句柄 最后add_summary即可写入 
    - 计算运行merge即可
    - 写入需要指定step num
- 利用tensorboard程序解析文件

## import

In [ ]:
import tensorflow as tf
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

## load data

In [ ]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

# tensorflow.Dataset.
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        # hstack vstack 用于合并二维数组变成矩阵形式
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        # 小心
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

## model defination

In [ ]:
tf.reset_default_graph() # 利用这个可清空defualt graph以及nodes

x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# conv1: 神经元图， feature_map, 输出图像
conv1_1 = tf.layers.conv2d(x_image,
                         32, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1_1')

conv1_2 = tf.layers.conv2d(conv1_1,
                         32, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv1_2')
# 16 * 16
pooling1 = tf.layers.max_pooling2d(conv1_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')


conv2_1 = tf.layers.conv2d(pooling1,
                         64, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv2_1')
conv2_2 = tf.layers.conv2d(conv2_1,
                         64, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv2_2')
# 8 * 8
pooling2 = tf.layers.max_pooling2d(conv2_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool2')

conv3_1 = tf.layers.conv2d(pooling2,
                         128, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv3_1')
conv3_2 = tf.layers.conv2d(conv3_1,
                         128, # output channel number
                         (3,3), # kernel size
                         padding = 'same',
                         activation = tf.nn.relu,
                         name = 'conv3_2')

# 4 * 4 * 32
pooling3 = tf.layers.max_pooling2d(conv3_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool3')
# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(tf.argmax(y_, 1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

## tensorboard 使用

 ### var defination
 - 自带batchsize维度不需要担心

In [ ]:
# 定义变量实现有效保存 可以指定保存内容名字
loss_summary = tf.summary.scalar('loss',loss)
# 'loss':[<loss,step>,<loss,steps>,...]
accuracy_summary = tf.summary.scalar('accuracy',accuracy)
# 自带batchsize维度不需要担心
source_image = (x_image+1)*127.5
image_summary = tf.summary.image('source_image',source_image)

def show_variables(var,name):
    """show variables in tensorboard"""
    """指定名字防止重复summary"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        min_summary = tf.summary.scalar('min',tf.reduce_min(var))
        max_summary = tf.summary.scalar('max',tf.reduce_max(var))
        mean_summary = tf.summary.scalar('mean',mean)
        stddev_summary = tf.summary.scalar('stddev',stddev)
        histogram_summary = tf.summary.histogram('histogram',var)

### 指定merge+dirpath

In [ ]:
# 调用
with tf.name_scope('summary_variables'):
    show_variables(conv1_1,'conv1_1')
    show_variables(conv1_2,'conv1_2')
    show_variables(conv2_1,'conv2_1')
    show_variables(conv2_2,'conv2_2')
    show_variables(conv3_1,'conv3_1')
    show_variables(conv3_2,'conv3_2')
# 指定merge
merge_summary_all = tf.summary.merge_all()
merge_summary_test = tf.summary.merge([loss_summary,accuracy_summary])
# filehandler
LOG_DIR = './run'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR,run_label)
train_log_dir = os.path.join(run_dir,'train')
test_log_dir = os.path.join(run_dir,'test')
if not os.path.exists(train_log_dir):
    os.makedirs(train_log_dir)
if not os.path.exists(test_log_dir):
    os.makedirs(test_log_dir)


### 指定filewriter+运行summary(merge)

In [ ]:
# cfg = tf.ConfigProto(allow_soft_placement=True )
# cfg.gpu_options.allow_growth = True
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100

# 由于summary耗时 指定轮次
output_summary_steps_num = 100
# train 10k: 71.35%
with tf.Session() as sess:
    sess.run(init)
    # filewriter
    # 可以指定sess.graph 或者不指定
    train_filewriter = tf.summary.FileWriter(train_log_dir,sess.graph)
    test_filewriter = tf.summary.FileWriter(test_log_dir)
    
    # 每次在test上想要选择同一批数据进行summary 在train时没有这种需求
    fixed_test_log_data,fixed_test_log_labels = test_data.next_batch(batch_size)
    
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        # 运行summary
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = (i+1) % output_summary_steps_num == 0
        if should_output_summary:
            eval_ops.append(merge_summary_all)
        
        eval_ops_result = sess.run(
            eval_ops,
            feed_dict={
                x: batch_data,
                y: batch_labels})
        
        loss_val,acc_val = eval_ops_result[0:2]
        # 写入 必须记得指定轮次
        if should_output_summary:
            train_summary_result = eval_ops_result[-1]
            train_filewriter.add_summary(train_summary_result,i+1)
            test_summary_result = sess.run(merge_summary_test,
                                          feed_dict = {
                                              x:fixed_test_log_data,
                                              y:fixed_test_log_labels
                                          })
            test_filewriter.add_summary(test_summary_result,i+1)
            
            
        if (i+1) % 50 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' 
                  % (i+1, loss_val, acc_val))
            
        if (i+1) % 200 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f'
                  % (i+1, test_acc))

### 调用命令解析+观察结果

In [ ]:
# 指定多个文件夹可以再显示的时候选择性显示
! tensorboard --logdir='test':./run/run_vgg_tensorboard/test,'train':./run/run_vgg_tensorboard/train